In [1]:
import json
import openai
import os
import re
from langchain_openai import ChatOpenAI
import transformers
from datasets import Dataset

In [19]:
file_path = 'C:\\Users\\PC\\Desktop\\DoYoung\\DS\\github\\bitamin_auto_readme_generator\\data\\object_detection\\output\\ocr_samples_txt\\webtoon_text.txt'

with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

In [20]:
api_key_filepath = "C:\\Users\\PC\\Desktop\\DoYoung\\DS\\비타민NLP_240701\\text_summarization\\openai_api_key.json"
with open(api_key_filepath, 'r') as f:
    api_key = json.load(f)
api_key = api_key['OPENAI_API_KEY']

In [21]:
os.environ['OPENAI_API_KEY'] = api_key

In [22]:
llm_3 = ChatOpenAI(
    model="gpt-3.5-turbo",
    openai_api_key=os.environ['OPENAI_API_KEY']
)

In [23]:
llm_4 = ChatOpenAI(
    model="gpt-4o",
    openai_api_key=os.environ['OPENAI_API_KEY']
)

### STEP1: 첫 5페이지에서 팀원, 제목, 목차 추출

In [24]:
end_marker = text.find('<p.6>')

if end_marker != -1:
    text5 = text[:end_marker]
else:
    text5 = text

In [25]:
def extract_info(text):
    prompt = f"""
    Extract the following information from the provided text:
    1. Name (excluding the team name, include all people listed)
    2. Title (include the entire title as it appears in the text)
    3. Main Topics (only main topics from the Table of Contents, excluding subtopics)

    The Main Topics should be extracted from the section that follows headers such as 'TABLE OF CONTENTS', '목차 소개', or any similar variation.
    Ensure that team member's name are not split into multiple lines. Name should be connected by commas if there are more than one.
    Ensure that title is not cut off and is extracted in their entirety.
    Ensure that main topics are not split into multiple lines. Name should be connected by commas if there are more than one.
    Exclude any subtopics or secondary information while extracting main topics.
    Stop at the next page marker '<p.'.

    Ensure that the output **never** starts with <xml>
    Do not include any additional notes or explanations in the output.

    Text: {text}


    Format the extracted information as follows:
    <subject>title</subject>
    <team>team members</team>
    <index>main topics</index>

    """

    response = llm_4.invoke(prompt)
    extracted_info = response.content.strip()
    return extracted_info

extracted_info = extract_info(text5)
print(extracted_info)


<title>24-1R 학기 프로젝트 독자와 웹툰 간 로그데이터를 활용한 웹툰 추천 시스템</title>
<team>강나영, 김나현, 엄성원, 이철민</team>
<index>프로젝트 소개, 데이터 수집 및 전처리, 모델 선택 및 학습, 웹툰 추천 시스템 구현, 결론 및 향후 과제</index>


In [26]:
main_topics_match = re.search(r'<index>(.*?)</index>', extracted_info, re.DOTALL)
if main_topics_match:
    main_topics = main_topics_match.group(1).strip()
    main_topics_list = [topic.strip() for topic in main_topics.split(',')]
else:
    main_topics_list = []

### STEP2: 전체 텍스트를 대주제 개수에 따라 구분

In [27]:
def remove_page(text):
    cleaned_text = re.sub(r'<p\.\d+>', '', text)
    return cleaned_text

In [28]:
cleaned_text = remove_page(text)
print(cleaned_text)


 BIT Amin
 24-1R 학기 프로젝트
 독자와 웹툰 간 로그데이터를 활용한 웹툰 추천 시스템
 20240605
 RecSys Team 2 - 강나영 김나현 엄성원 이철민

 BIT Amin
 TABLE OF CONTENTS
 목차소개
 이
 프로젝트 소개 프로젝트 배경 프로젝트 목표
 02
 데이터 수집 및 전처리 데이터 소스 설명 데이터 전처리
 03
 모델 선택 및 학습 모델 비교 최종 모델 선정
 04
 웹툰 추천 시스템 구현 기존 사용자 신규 사용자
 05
 결론 및 향후 과제 프로젝트 요약 및 의의 한계점 및 향후 과제

 이
 프로젝트 소개
 BITAmir
 프로젝트 배경
 정제된 '대량의 데이터'를 사용하여 모델 선택 범위를 넓히고자 함 사용자의 데이터를 효과적으로 사용할 수 있는 주제 선정 직관적으로 추천 시스템의 작동을 확인할 수 있도록 가급적 국내 최신 데이터 활용
 kaggle
 프로젝트 목표
 사용자 데이터 웹툰 데이터를 이용해 개인화된 추천 시스템 개발 사용자가 흥미를 느낄 만한 웹툰을 정확히 추천 사용자와 아이템의 데이터에 맞는 알고리즘 nteraction 탐색 및 적용

 02
 데이터 수집 및 전처리
 BITAmir
 데이터 소스 설명
 Webtoon_CustomersCsV
 Webtoon_articlescs!
 Webtoon_interaction_traincs!
 interaction data = pd read _Csv< Idata/webtoon interaction webtoon_data pdread_csv /data/webtoon = user_data pdread = I_csv[ /data/webtoon CustomersCsy'

 02
 데이터 수집 및 전처리
 BITAmin
 데이터 전처리
 # 649만 개의 데이터 100개 이상 평가한 유저만 남음

 02
 데이터 수집 및 전처리
 BIT Amin
 데이터 전처리
 피봇 테이블 형식의 데이터 생성

 03
 모델 선택 학습 및 CF
 BIT Amin
 Filte

In [29]:
def divide_text(extracted_info, main_topics_list, text):
    prompt = f"""
    The provided text contains specific sections that need to be excluded before dividing it according to the main topics.
    Exclude the following sections from the text:
    {extracted_info}

    After exclusion, divide the provided text according to the main topics.

    The text should be divided into exactly {len(main_topics_list)} sections, separated by "===================================".

    Use the main topics provided below as the indices for division:
    {main_topics_list}

    The text should remain unmodified.
    The text should only be partitioned according to the number of main topics.

    Text: {text}
    """

    response = llm_4.invoke(prompt)
    divided_text = response.content.strip()
    return divided_text


divided_text = divide_text(extracted_info, main_topics_list, cleaned_text)
print(divided_text)


프로젝트 소개
 BIT Amin
 24-1R 학기 프로젝트
 독자와 웹툰 간 로그데이터를 활용한 웹툰 추천 시스템
 20240605
 RecSys Team 2 - 강나영 김나현 엄성원 이철민

 BIT Amin
 TABLE OF CONTENTS
 목차소개
 이
 프로젝트 소개 프로젝트 배경 프로젝트 목표
 02
 데이터 수집 및 전처리 데이터 소스 설명 데이터 전처리
 03
 모델 선택 및 학습 모델 비교 최종 모델 선정
 04
 웹툰 추천 시스템 구현 기존 사용자 신규 사용자
 05
 결론 및 향후 과제 프로젝트 요약 및 의의 한계점 및 향후 과제

 이
 프로젝트 소개
 BITAmir
 프로젝트 배경
 정제된 '대량의 데이터'를 사용하여 모델 선택 범위를 넓히고자 함 사용자의 데이터를 효과적으로 사용할 수 있는 주제 선정 직관적으로 추천 시스템의 작동을 확인할 수 있도록 가급적 국내 최신 데이터 활용
 kaggle
 프로젝트 목표
 사용자 데이터 웹툰 데이터를 이용해 개인화된 추천 시스템 개발 사용자가 흥미를 느낄 만한 웹툰을 정확히 추천 사용자와 아이템의 데이터에 맞는 알고리즘 nteraction 탐색 및 적용
데이터 수집 및 전처리
 02
 데이터 수집 및 전처리
 BITAmir
 데이터 소스 설명
 Webtoon_CustomersCsV
 Webtoon_articlescs!
 Webtoon_interaction_traincs!
 interaction data = pd read _Csv< Idata/webtoon interaction webtoon_data pdread_csv /data/webtoon = user_data pdread = I_csv[ /data/webtoon CustomersCsy'

 02
 데이터 수집 및 전처리
 BITAmin
 데이터 전처리
 # 649만 개의 데이터 100개 이상 평가한 유저만 남음

 02
 데이터 수집 및 전처리
 BIT Amin
 데이터 전처리
 피봇 테이블 형식의 데이터 생성
모델 선택 및 학습
 03
 모

### STEP3: 요약

In [30]:
def summarize(text, main_topics_list):
    prompt = f"""
    The provided text is divided into several partitions separated by "===================================".
    Summarize each partition separately. 
    Do not change the language of the original input text. 
    Summarize in the same language as the input text.
    The purpose of the summaries is to create a comprehensive README for a GitHub project.

    Ensure that each summary accurately captures the key points and essential information of its respective partition.
    Summarize in a way that is concise and informative, omitting repetitive mentions of the main topic after the first mention in each partition.
    Focus on the key details and results relevant to the project.

    Do not remove or modify the "===================================" separators.
    Use the following main topics to guide the summaries:
    {main_topics_list}
    
    The summaries should follow this format:
    ===================================
    Main topic
    - subtopic: detailed contents
    - subtopic: detailed contents
    ===================================
    Main topic
    - subtopic: detailed contents
    - subtopic: detailed contents
    ===================================

    Here is an example of a well-structured summary:

    ===================================
    프로젝트 소개
    - 프로젝트 배경: 정제된 '대량의 데이터'를 사용하여 모델 선택 범위를 넓히고자 함.
    - 프로젝트 목표: 사용자 데이터, 웹툰 데이터를 이용해 개인화된 추천 시스템 개발. 사용자와 아이템의 interaction 데이터에 맞는 알고리즘 탐색 및 적용.
    ===================================
    데이터 수집 및 전처리
    - 데이터 소스 설명: 다양한 데이터 소스 사용. 데이터 전처리 과정 설명.
    - 데이터 처리: 피봇 테이블 형식의 데이터 생성, 데이터 정제 및 전처리.
    ===================================

    Text: {text}
    """

    response = llm_4.invoke(prompt)
    result = response.content.strip()
    return result

In [31]:
summarized_text = summarize(divided_text, main_topics_list)
print(summarized_text)

프로젝트 소개
- 프로젝트 배경: 정제된 '대량의 데이터'를 사용하여 모델 선택 범위를 넓히고자 함. 사용자의 데이터를 효과적으로 사용할 수 있는 주제를 선정하고, 직관적으로 추천 시스템의 작동을 확인할 수 있도록 국내 최신 데이터를 활용.
- 프로젝트 목표: 사용자 데이터와 웹툰 데이터를 이용해 개인화된 추천 시스템을 개발하여 사용자가 흥미를 느낄 만한 웹툰을 정확히 추천. 사용자와 아이템의 interaction 데이터에 맞는 알고리즘 탐색 및 적용.
데이터 수집 및 전처리
- 데이터 소스 설명: Webtoon_Customers.csv, Webtoon_articles.csv, Webtoon_interaction_train.csv 데이터 파일 사용.
- 데이터 전처리: 649만 개의 데이터 중 100개 이상 평가한 유저만 남기고, 피봇 테이블 형식의 데이터 생성.
모델 선택 및 학습
- CF (Collaborative Filtering) 모델: 사용자 유사도를 측정하는 User-Based와 아이템 간 유사도를 측정하는 Item-Based로 나뉨. 데이터 크기와 정보에 따라 UBCF 또는 IBCF가 적합. 장점: 직관적인 결과. 단점: Cold Start Problem, 계산량 많음.
- NCF (Neural Collaborative Filtering) 모델: 사용자와 아이템을 embedding vector로 표현해 interaction을 학습. 장점: 복잡한 비선형 학습 가능. 단점: Long tail, Data Sparsity.
- SVD (Singular Value Decomposition) 모델: 사용자-아이템 행렬을 세 개의 행렬로 분해해 잠재 요인을 추출. 장점: 차원 축소로 효율성 증가. 단점: 큰 데이터셋에서 계산 비용 증가.
- ALS (Alternating Least Squares) 모델: 행렬 분해 기법으로 사용자-아이템 상호작용 행렬을 두 개의 저차원 행렬로 분해해 잠재 요인 학습. 대규모 데이터셋에 적합.
- 최종 모델 선정: ALS 모델

### STEP4: 마크다운 형식으로 변환

In [32]:
def tag_text(text):
    prompt = f"""
    Tag each part of the text according to the following rules:
    - The first line before any "-" should be tagged as <main>.
    - Each line starting with "-" should have the part before ":" tagged as <sub> and the part after ":" tagged as <content>.
    - Remove "-", "=", and ":" characters from the text.
    - Ensure the tags are correctly closed and formatted.
    Ensure that the output **never** starts with <xml>

    Here is an example of how the tagged text should look:

    <main>대한민국의 계절</main>
    <sub>여름</sub> <content>요즘 한국의 여름은 매우 덥고 습한 편이다.</content>
    <sub>겨울</sub> <content>12월부터 온도가 급격히 떨어지며 건조해진다.</content>

    <main>대한민국의 휴일</main>
    <sub>추석</sub> <content>추석은 비교적 휴일 기간이 긴 편이다.</content>
    <sub>설날</sub> <content>설날에는 떡국을 먹으며, 가족들과 인사를 나눈다.</content>

    Text:
    {text}
    """

    #temperature 설정
    response = llm_4.invoke(prompt)
    result = response.content.strip()
    return result


In [33]:
tage_result = tag_text(summarized_text)
print(tage_result)

<main>프로젝트 소개</main>
<sub>프로젝트 배경</sub> <content>정제된 '대량의 데이터'를 사용하여 모델 선택 범위를 넓히고자 함. 사용자의 데이터를 효과적으로 사용할 수 있는 주제를 선정하고, 직관적으로 추천 시스템의 작동을 확인할 수 있도록 국내 최신 데이터를 활용.</content>
<sub>프로젝트 목표</sub> <content>사용자 데이터와 웹툰 데이터를 이용해 개인화된 추천 시스템을 개발하여 사용자가 흥미를 느낄 만한 웹툰을 정확히 추천. 사용자와 아이템의 interaction 데이터에 맞는 알고리즘 탐색 및 적용.</content>

<main>데이터 수집 및 전처리</main>
<sub>데이터 소스 설명</sub> <content>Webtoon_Customers.csv, Webtoon_articles.csv, Webtoon_interaction_train.csv 데이터 파일 사용.</content>
<sub>데이터 전처리</sub> <content>649만 개의 데이터 중 100개 이상 평가한 유저만 남기고, 피봇 테이블 형식의 데이터 생성.</content>

<main>모델 선택 및 학습</main>
<sub>CF (Collaborative Filtering) 모델</sub> <content>사용자 유사도를 측정하는 User-Based와 아이템 간 유사도를 측정하는 Item-Based로 나뉨. 데이터 크기와 정보에 따라 UBCF 또는 IBCF가 적합. 장점: 직관적인 결과. 단점: Cold Start Problem, 계산량 많음.</content>
<sub>NCF (Neural Collaborative Filtering) 모델</sub> <content>사용자와 아이템을 embedding vector로 표현해 interaction을 학습. 장점: 복잡한 비선형 학습 가능. 단점: Long tail, Data Sparsity.</content>
<sub>SVD (Singular Value Decomposition) 모